### Problem with HMI
    The get_active_region_labels function identifies active regions and labels each pixel 
    in that active regions with a number. This function takes between 4 - 7 minutes to label
    an MDI image. For HMI, however, the process took over 2 hours before I stopped it. 
    This function needs to be made faster or be replaced, for HMI images. 

In [1]:
import astropy
from astropy.io import fits
from math import pi, sin, cos, sqrt, atan2
from time import time
import numpy as np
import pandas as pd
import os
import json
from collections import Counter,defaultdict
import astropy.convolution
from astropy.convolution import convolve_fft, convolve
from time import time
from scipy import ndimage as ndi
from skimage.morphology import watershed
from skimage.feature import peak_local_max
import pickle

import extract_image_features 
import Centroid_Labeling
from sunspot_feature_extraction import extract_features
from Cylindrical_Map_Transformation import get_header_params_MDI,get_header_params_HMI, map_disk_cylindric

#### Load HMI (and MDI) Images

In [2]:
image_path = "/Users/Alexander/NASA/NASA_Sample_Data/HMI_Alexander/"

In [3]:
file_name_list = []
for filename in os.listdir(image_path):
    if "ds_store" not in filename.lower():
        file_name_list.append(filename)

In [4]:
hmi_file = file_name_list[9]
hmi_file

'HMI.m2011.01.01_00_00_00.fits'

In [11]:
hdulist = fits.open(image_path + hmi_file)

In [13]:
hdulist[1].header

 [astropy.io.fits.verify]


XTENSION= 'IMAGE   '           / IMAGE extension                                
BITPIX  =                   32 / number of bits per data pixel                  
NAXIS   =                    2 / number of data axes                            
NAXIS1  =                 4096 / length of data axis 1                          
NAXIS2  =                 4096 / length of data axis 2                          
PCOUNT  =                    0 / required keyword; must = 0                     
GCOUNT  =                    1 / required keyword; must = 1                     
DATE    = '2011-02-02T01:50:19'                                                 
DATE-OBS= '2010-12-31T23:58:10.20'                                              
TELESCOP= 'SDO/HMI'                                                             
INSTRUME= 'HMI_SIDE1'                                                           
WAVELNTH= 6173.0                                                                
CAMERA  =                   

In [42]:
def preprocess_image(hdu_object):
    # upload data to dataframe in order to replace nans with zeros
    df = pd.DataFrame(hdu_object.data)
    df.fillna(value=0,
              inplace=True)
    clean_data = df.values

    # choice appropriate method base on file type
    if "MDI" in hdu_object.header["INSTRUME"]:
        params_method = get_header_params_MDI
    elif "HMI" in hdu_object.header["INSTRUME"]:
        params_method = get_header_params_HMI
        print "get_header_params_HMI"
    else:
        print "ERROR: no INSTRUME param located in header!"
    # extract parameters from header  
    xCen,yCen,s0,nx,pixsize,p0,b0, r0 = params_method(hdu_object.header)

    # transform data into cylindrical equal area map
    print "running map_disk_cylindric...."
    return map_disk_cylindric(xCen,yCen,s0,nx,pixsize,p0,b0, r0, clean_data)

In [43]:
def get_active_region_map(path, image_file, mdi_flux_filter, hmi_flux_filter, kernal_std):

    if "HMI" in image_file:
        flux_magnitude_filter = hmi_flux_filter
        kernal_std = kernal_std * 4
        hdu_index = 1
        print "hmi image..."
    else:
        hdu_index = 0
        flux_magnitude_filter = mdi_flux_filter
        print "mdi image..."

    gauss = astropy.convolution.Gaussian2DKernel(stddev=kernal_std)
    hdulist = fits.open(path + image_file)
    hdu = hdulist[hdu_index] # <-- NEED CASE CONDITIONAL FOR MDI/HMI
    print "processing_image ..."
    clean_data = preprocess_image(hdu)
    data_abs = np.abs(clean_data)
    # smooth data with Fast Fourier Transform
    smoothing = convolve_fft(data_abs, gauss)
    smoothing[smoothing < flux_magnitude_filter] = 0.
    smoothing[smoothing >= flux_magnitude_filter] = 1 
    return np.where(smoothing !=0.0 , clean_data, smoothing), hdu

In [22]:
df = pd.DataFrame(hdulist[1].data)
df.fillna(value=0,
          inplace=True)
clean_data = df.values

In [23]:
clean_data

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [27]:
xCen,yCen,s0,nx,pixsize,p0,b0, r0 = get_header_params_HMI(hdulist[1].header)
xCen,yCen,s0,nx,pixsize,p0,b0, r0

(2039.383545,
 2040.383545,
 975.936584,
 4096,
 0.504275,
 -180.082565,
 -2.974446,
 1935.3261295919885)

In [40]:
start = time()
test_data = map_disk_cylindric(xCen,yCen,s0,nx,pixsize,p0,b0, r0, clean_data)
end = time()
print "Time Elapsed = {:.4} ".format((end - start)/60)

Time Elapsed = 6.445 


In [41]:
test_data

array([[ -2.71826519,  -2.71111827,  -2.70384613, ...,   2.41746827,
          2.41237654,   2.40723531],
       [ 10.7529    ,  10.76273139,  10.77254512, ...,  -3.81826782,
         -3.8487366 ,  -3.87895189],
       [ -0.27818778,  -0.27647357,  -0.27444449, ...,   0.96192477,
          0.94175702,   0.9204597 ],
       ..., 
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ]])

In [33]:
gauss = astropy.convolution.Gaussian2DKernel(stddev=kernal_std * 4)
smoothing = convolve_fft(test_data, gauss)

In [34]:
smoothing

array([[  2.88496856e-01,   2.98374589e-01,   3.08430202e-01, ...,
         -9.68277272e-02,  -9.61087679e-02,  -9.52876020e-02],
       [  2.89098435e-01,   2.99069392e-01,   3.09223839e-01, ...,
         -9.75620937e-02,  -9.68950521e-02,  -9.61216042e-02],
       [  2.89387504e-01,   2.99443712e-01,   3.09689033e-01, ...,
         -9.81266750e-02,  -9.75175017e-02,  -9.67976302e-02],
       ..., 
       [  1.49880108e-15,   1.99840144e-15,   2.10942375e-15, ...,
          2.15105711e-16,  -5.13478149e-16,  -1.11022302e-16],
       [  1.63757896e-15,   2.13717932e-15,   2.27595720e-15, ...,
          1.45716772e-16,  -6.17561557e-16,  -2.42861287e-16],
       [  1.60982339e-15,   2.10942375e-15,   2.22044605e-15, ...,
          3.46944695e-16,  -4.09394740e-16,  -6.24500451e-17]])

In [44]:
start = time()
mdi_flux_filter = 120
hmi_flux_filter = 100 
kernal_std = 8
image, hdu= get_active_region_map(image_path, hmi_file,  mdi_flux_filter , hmi_flux_filter, kernal_std)
end = time()
print "Time Elapsed = {:.4}".format((end - start)/60)

hmi image...
processing_image ...
get_header_params_HMI
running map_disk_cylindric....
Time Elapsed = 7.175


In [53]:
def get_active_region_labels(image):
    # Exact euclidean distance transform
    print "get distance..."
    distance = ndi.distance_transform_edt(image)
    # Find peaks in an image, and return them as coordinates or a boolean array
    print "get local_maxi..."
    local_maxi = peak_local_max(distance, 
                                indices=False, 
                                footprint=np.ones((40, 40)),
                                labels=image)
    # return labels of peaks
    print "get markers..."
    markers = ndi.label(local_maxi)[0]
    # return active region labels 
    print "get watershed..."
    return watershed(-distance, markers, mask=image)

In [54]:
start = time()
labels = get_active_region_labels(image)
end = time()
print "Time Elapsed = {:.4}".format((end - start)/60)

get distance...
get local_maxi...


KeyboardInterrupt: 